In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations, islice
from sklearn.decomposition import FastICA
from concurrent.futures import ThreadPoolExecutor, as_completed
from numba import njit
import cupy as cp

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# --- Workaround for torch_geometric compatibility ---
if not hasattr(torch.serialization, 'add_safe_globals'):
    torch.serialization.add_safe_globals = lambda x: None

from torch_geometric.nn import GATConv
from sklearn.metrics import r2_score
from sklearn.metrics.pairwise import cosine_similarity

# ===================== Propagation Graph Implementation =====================
@njit
def cost(a, b):
    return abs(a - b)

@njit
def compute_d_matrix(s, t):
    T = s.shape[0]
    D = np.empty((T, T), dtype=np.float64)
    D[0, 0] = cost(s[0], t[0])
    for j in range(1, T):
        D[0, j] = D[0, j-1] + cost(s[0], t[j])
    for i in range(1, T):
        D[i, 0] = D[i-1, 0] + cost(s[i], t[0])
    for i in range(1, T):
        for j in range(1, T):
            a_val = D[i-1, j]
            b_val = D[i, j-1]
            c_val = D[i-1, j-1]
            D[i, j] = min(a_val, b_val, c_val) + cost(s[i], t[j])
    return D

@njit
def compute_F_matrix(s, t, D):
    T = s.shape[0]
    F = np.zeros((T, T), dtype=np.float64)
    F[0, 0] = 1.0
    for i in range(T):
        for j in range(T):
            if i == 0 and j == 0:
                continue
            total = 0.0
            c = cost(s[i], t[j])
            if i > 0 and abs(D[i, j] - (D[i-1, j] + c)) < 1e-8:
                total += F[i-1, j]
            if j > 0 and abs(D[i, j] - (D[i, j-1] + c)) < 1e-8:
                total += F[i, j-1]
            if i > 0 and j > 0 and abs(D[i, j] - (D[i-1, j-1] + c)) < 1e-8:
                total += F[i-1, j-1]
            F[i, j] = total
    return F

@njit
def compute_B_matrix(s, t, D):
    T = s.shape[0]
    B = np.zeros((T, T), dtype=np.float64)
    B[T-1, T-1] = 1.0
    for i in range(T-1, -1, -1):
        for j in range(T-1, -1, -1):
            if i == T-1 and j == T-1:
                continue
            total = 0.0
            if i+1 < T and abs(D[i+1, j] - (D[i, j] + cost(s[i+1], t[j]))) < 1e-8:
                total += B[i+1, j]
            if j+1 < T and abs(D[i, j+1] - (D[i, j] + cost(s[i], t[j+1]))) < 1e-8:
                total += B[i, j+1]
            if i+1 < T and j+1 < T and abs(D[i+1, j+1] - (D[i, j] + cost(s[i+1], t[j+1]))) < 1e-8:
                total += B[i+1, j+1]
            B[i, j] = total
    return B

@njit
def avg_time_delay(s, t):
    T = s.shape[0]
    D = compute_d_matrix(s, t)
    F = compute_F_matrix(s, t, D)
    B = compute_B_matrix(s, t, D)
    total_delay = 0.0
    for i in range(1, T):
        for j in range(1, T):
            c = cost(s[i], t[j])
            if abs(D[i, j] - (D[i-1, j-1] + c)) < 1e-8:
                total_delay += (j - i) * F[i-1, j-1] * B[i, j]
    num_alignments = B[0, 0]
    if num_alignments == 0:
        return 0.0
    return total_delay / num_alignments

def avg_time_delay_gpu(s, t):
    s_gpu = cp.asarray(s)
    t_gpu = cp.asarray(t)
    T = s_gpu.shape[0]
    D_gpu = cp.zeros((T, T), dtype=cp.float64)
    D_gpu[0, 0] = cp.abs(s_gpu[0] - t_gpu[0])
    for j in range(1, T):
        D_gpu[0, j] = D_gpu[0, j-1] + cp.abs(s_gpu[0] - t_gpu[j])
    for i in range(1, T):
        D_gpu[i, 0] = D_gpu[i-1, 0] + cp.abs(s_gpu[i] - t_gpu[0])
    for i in range(1, T):
        for j in range(1, T):
            D_gpu[i, j] = cp.minimum(cp.minimum(D_gpu[i-1, j], D_gpu[i, j-1]),
                                       D_gpu[i-1, j-1]) + cp.abs(s_gpu[i] - t_gpu[j])
    D = cp.asnumpy(D_gpu)
    return avg_time_delay(s, t)

def build_propagation_graph(signal_list):
    """
    Construct a propagation graph from a list of stock signals.
    Each signal is a 1D numpy array.
    Returns:
        graph: dict mapping stock index to list of neighbor indices.
        delays: dict mapping (i, j) to average delay.
    """
    N = len(signal_list)
    edges = []
    delays = {}
    for i in range(N):
        for j in range(i+1, N):
            delay_val = avg_time_delay(np.array(signal_list[i]), np.array(signal_list[j]))
            if np.isclose(delay_val, 0.0):
                continue
            if delay_val > 0:
                edges.append((i, j))
                delays[(i, j)] = delay_val
            else:
                edges.append((j, i))
                delays[(j, i)] = -delay_val
    graph = {}
    for (u, v) in edges:
        graph.setdefault(u, []).append(v)
    return graph, delays

def draw_graph(graph, delays, title="Propagation Graph", filename="graph.png"):
    G = nx.DiGraph()
    for u, vs in graph.items():
        for v in vs:
            G.add_edge(u, v, weight=delays.get((u, v), 0))
    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(10, 8))
    nx.draw(G, pos, with_labels=True, node_color="lightblue", arrowstyle="->", arrowsize=20)
    edge_labels = {(u, v): f"{data['weight']:.2f}" for u, v, data in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    plt.title(title)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

def convert_prop_graph_to_edge_tensors(delays):
    """
    Convert the delays dictionary into edge_index and edge_weight tensors.
    We use exp(-delay) as weight.
    """
    edges = []
    weights = []
    for (u, v), delay in delays.items():
        edges.append([u, v])
        weights.append(np.exp(-delay))
    edge_index = torch.tensor(np.array(edges).T, dtype=torch.long)
    edge_weight = torch.tensor(weights, dtype=torch.float)
    return edge_index, edge_weight

# ===================== Feature Engineering & Dataset =====================
def compute_features(prices, window=5):
    """
    Compute log return, rolling mean, and rolling std.
    Returns a DataFrame with MultiIndex columns (stock, feature).
    """
    log_ret = np.log(prices) - np.log(prices.shift(1))
    rolling_mean = log_ret.rolling(window=window).mean()
    rolling_std = log_ret.rolling(window=window).std()
    
    arrays = []
    for stock in prices.columns:
        for feat in ['log', 'mean', 'std']:
            arrays.append((stock, feat))
    col_index = pd.MultiIndex.from_tuples(arrays, names=['stock', 'feature'])
    feature_df = pd.DataFrame(index=prices.index, columns=col_index)
    for stock in prices.columns:
        feature_df[(stock, 'log')] = log_ret[stock]
        feature_df[(stock, 'mean')] = rolling_mean[stock]
        feature_df[(stock, 'std')] = rolling_std[stock]
    feature_df = feature_df.dropna()
    return feature_df

class StockFeatureDataset(Dataset):
    def __init__(self, feature_df, window_size):
        """
        feature_df: DataFrame with MultiIndex columns (stock, feature)
        window_size: number of historical days.
        For each sample, for each stock, extract a window of features (shape: (window_size, num_features)),
        flatten it, and use as input.
        Target: next-day log return for each stock.
        """
        self.feature_df = feature_df
        self.window_size = window_size
        self.stocks = feature_df.columns.levels[0].tolist()
        self.features = feature_df.columns.levels[1].tolist()
        self.num_features = len(self.features)
        self.num_stocks = len(self.stocks)
        self.num_samples = feature_df.shape[0] - window_size
    def __len__(self):
        return self.num_samples
    def __getitem__(self, idx):
        X_list = []
        targets = []
        for stock in self.stocks:
            stock_data = self.feature_df[stock].iloc[idx: idx + self.window_size].values
            X_list.append(stock_data.flatten())
            targets.append(self.feature_df[stock].iloc[idx + self.window_size]['log'])
        X = np.vstack(X_list)
        y = np.array(targets)
        return torch.tensor(X, dtype=torch.float), torch.tensor(y, dtype=torch.float)

# ===================== KAN Model Definition =====================
class StockKAN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=4):
        super(StockKAN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )
        self.gat = GATConv(hidden_dim, hidden_dim, heads=heads, concat=False)
        self.predictor = nn.Sequential(
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x, edge_index, edge_weight=None):
        x = self.encoder(x)
        x = self.gat(x, edge_index)
        out = self.predictor(x)
        return out

# ===================== Backtesting Function for All Stocks =====================
def backtest_all_stocks(model, test_dataset, edge_index_prop, edge_weight_prop, device, output_folder):
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for X, y in test_loader:
            X = X.to(device)
            y = y.to(device)
            batch_size, num_stocks, input_dim = X.shape
            X_reshaped = X.view(-1, input_dim)
            out = model(X_reshaped, edge_index_prop, edge_weight_prop)
            out = out.view(batch_size, num_stocks, -1)
            predictions.append(out.squeeze().cpu().numpy())
            actuals.append(y.cpu().numpy())
    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)
    
    portfolio_metrics = ""
    # Generate and save PnL and portfolio statistics for each stock
    for stock_idx, stock_name in enumerate(test_dataset.stocks):
        stock_preds = predictions[:, stock_idx]
        stock_actuals = actuals[:, stock_idx]
        rmse = np.sqrt(np.mean((stock_preds - stock_actuals)**2))
        r2 = r2_score(stock_actuals, stock_preds)
        metrics_str = f"Stock: {stock_name}\n  RMSE: {rmse:.6f}\n  R^2: {r2:.6f}\n"
        print(f"Backtest metrics for {stock_name}:")
        print(metrics_str)
        
        # Correlation scatter plot
        plt.figure(figsize=(8, 6))
        plt.scatter(stock_actuals, stock_preds, alpha=0.6)
        plt.xlabel("Actual Returns")
        plt.ylabel("Predicted Returns")
        plt.title(f"Correlation Scatter for {stock_name} (Backtest)")
        plt.grid(True)
        plt.tight_layout()
        corr_filename = os.path.join(output_folder, f"backtest_correlation_{stock_name}.png")
        plt.savefig(corr_filename)
        plt.close()
        print(f"Saved backtest correlation plot to {corr_filename}")
        
        # Portfolio metrics
        mean_return = np.mean(stock_actuals)
        std_return = np.std(stock_actuals)
        annualized_vol = std_return * np.sqrt(252)
        sharpe_ratio = (mean_return / std_return) * np.sqrt(252) if std_return != 0 else 0.0
        metrics_str += (f"  Mean Daily Return: {mean_return:.6f}\n" +
                        f"  Daily Return Std Dev: {std_return:.6f}\n" +
                        f"  Annualized Volatility: {annualized_vol:.6f}\n" +
                        f"  Sharpe Ratio: {sharpe_ratio:.6f}\n\n")
        print(f"Backtest Portfolio Metrics for {stock_name}:\n"
              f"  Mean Daily Return: {mean_return:.6f}\n"
              f"  Daily Return Std Dev: {std_return:.6f}\n"
              f"  Annualized Volatility: {annualized_vol:.6f}\n"
              f"  Sharpe Ratio: {sharpe_ratio:.6f}")
        
        # Plot cumulative PnL
        positions = np.sign(stock_preds)
        daily_pnl = positions * stock_actuals
        cumulative_pnl = np.cumsum(daily_pnl)
        plt.figure(figsize=(10, 5))
        plt.plot(cumulative_pnl, marker='o', linestyle='-', label="Cumulative PnL")
        plt.xlabel("Test Sample Index (Days)")
        plt.ylabel("Cumulative PnL")
        plt.title(f"PnL Chart for {stock_name} (Backtest)")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        pnl_filename = os.path.join(output_folder, f"backtest_pnl_{stock_name}.png")
        plt.savefig(pnl_filename)
        plt.close()
        print(f"Saved backtest PnL chart to {pnl_filename}")
        
        portfolio_metrics += metrics_str

    # Save portfolio metrics to a text file in the graphs folder
    metrics_file = os.path.join(output_folder, "portfolio_metrics.txt")
    with open(metrics_file, "w") as f:
        f.write(portfolio_metrics)
    print(f"Saved portfolio metrics to {metrics_file}")

# ===================== Main Pipeline =====================
def main():
    # Load price data
    df = pd.read_csv('./data/global_titans.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    stocks = df.columns.tolist()
    
    # Compute features: log return, rolling mean, and rolling std (window=5)
    features_df = compute_features(df, window=5)
    
    # Split into training and testing sets (first half for training, second half for testing)
    T = features_df.shape[0]
    train_features = features_df.iloc[:T//2]
    test_features = features_df.iloc[T//2:]
    
    # For propagation graph, use training log returns (from features)
    train_returns = train_features.xs('log', level='feature', axis=1)
    train_data = train_returns.values
    n_components = min(3, train_data.shape[1])
    ica = FastICA(n_components=n_components, random_state=0, max_iter=100, tol=1e-2)
    try:
        ica.fit(train_data)
    except Exception as e:
        print("FastICA failed on training data:", e)
        return
    
    # Build propagation graph using the provided implementation:
    # Process data in windows (prop_window_size=30) over entire dataset
    prop_window_size = 30
    num_windows = df.shape[0] // prop_window_size
    max_triplets = 1000
    all_window_avgs = []
    
    def process_window_prop(w):
        window_df = df.iloc[w*prop_window_size : (w+1)*prop_window_size]
        pair_delays = {}
        for triplet in islice(combinations(stocks, 3), max_triplets):
            data = window_df[list(triplet)].values  # shape: (prop_window_size, 3)
            n_components_local = 3
            ica_local = FastICA(n_components=n_components_local, random_state=0, max_iter=100, tol=1e-2)
            try:
                S = ica_local.fit_transform(data)
            except Exception:
                print("Failed to converge")
                continue
            A = ica_local.mixing_
            best_comp = None
            best_spread = np.inf
            for k in range(n_components_local):
                spread = np.max(A[:, k]) - np.min(A[:, k])
                if spread < best_spread:
                    best_spread = spread
                    best_comp = k
            rec_signals = {}
            for idx, stock in enumerate(triplet):
                rec_signal = A[idx, best_comp] * S[:, best_comp]
                rec_signals[stock] = rec_signal
            for stock1, stock2 in combinations(triplet, 2):
                delay_val = avg_time_delay(rec_signals[stock1], rec_signals[stock2])
                if np.isclose(delay_val, 0.0):
                    continue
                if delay_val > 0:
                    key = (stock1, stock2)
                    val = delay_val
                else:
                    key = (stock2, stock1)
                    val = -delay_val
                pair_delays.setdefault(key, []).append(val)
        window_avg = {pair: np.mean(vals) for pair, vals in pair_delays.items()}
        return window_avg
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = {executor.submit(process_window_prop, w): w for w in range(num_windows)}
        for future in as_completed(futures):
            try:
                result = future.result()
                all_window_avgs.append(result)
            except Exception as e:
                print("A window failed:", e)
    
    # Aggregate delays over windows
    aggregate_delays = {}
    for window_avg in all_window_avgs:
        for pair, delay_val in window_avg.items():
            if pair in aggregate_delays:
                s, cnt = aggregate_delays[pair]
                aggregate_delays[pair] = (s + delay_val, cnt + 1)
            else:
                aggregate_delays[pair] = (delay_val, 1)
    cumulative_avg = {pair: s/cnt for pair, (s, cnt) in aggregate_delays.items()}
    
    # Build propagation graph dictionary from cumulative delays
    prop_graph = {stock: [] for stock in stocks}
    for (u, v), avg_delay in cumulative_avg.items():
        prop_graph.setdefault(u, []).append(v)
    
    # Draw and save the final propagation graph and delays CSV
    output_folder = "graphs"
    os.makedirs(output_folder, exist_ok=True)
    final_graph_file = os.path.join(output_folder, "final_cumulative_prop_graph.png")
    draw_graph(prop_graph, cumulative_avg, title="Final Cumulative Propagation Graph", filename=final_graph_file)
    print(f"Saved final propagation graph to {final_graph_file}")
    final_df = pd.DataFrame([(u, v, avg_delay) for (u, v), avg_delay in cumulative_avg.items()],
                            columns=["Stock1", "Stock2", "AvgDelay"])
    final_df.to_csv(os.path.join(output_folder, "final_cumulative_delays.csv"), index=False)
    print("Saved cumulative propagation delays CSV.")
    
    # Convert propagation graph delays into edge tensors.
    def convert_prop_graph_to_edge_tensors(delays):
        edges = []
        weights = []
        for (u, v), delay in delays.items():
            edges.append([stocks.index(u), stocks.index(v)])
            weights.append(np.exp(-delay))
        edge_index = torch.tensor(np.array(edges).T, dtype=torch.long)
        edge_weight = torch.tensor(weights, dtype=torch.float)
        return edge_index, edge_weight

    edge_index_prop, edge_weight_prop = convert_prop_graph_to_edge_tensors(cumulative_avg)
    print("Propagation graph edges (tensor):", edge_index_prop.shape[1])
    
    # Create datasets using the multi-feature DataFrame
    window_size = 60
    train_dataset = StockFeatureDataset(train_features, window_size)
    test_dataset = StockFeatureDataset(test_features, window_size)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
    
    # Input dimension = window_size * number of features (3)
    input_dim = window_size * 3
    model = StockKAN(input_dim=input_dim, hidden_dim=64, output_dim=1, heads=4)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    model.to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    # --- Training improvements for speed:
    # 1. Process entire batches at once.
    # 2. Move edge tensors to the device outside the loop.
    edge_index_prop = edge_index_prop.to(device)
    edge_weight_prop = edge_weight_prop.to(device)
    
    epochs = 50
    model.train()
        
    # Initialize AMP GradScaler for mixed precision training
    scaler = torch.amp.GradScaler()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                batch_size, num_stocks, input_dim = X.shape
                X_reshaped = X.view(-1, input_dim)
                out = model(X_reshaped, edge_index_prop, edge_weight_prop)
                out = out.view(batch_size, num_stocks, -1)
                loss = criterion(out.squeeze(), y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.6f}")
    
    # Evaluate on test set (ensuring no training data is used)
    model.eval()
    predictions = []
    actuals = []
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)
    with torch.no_grad():
        for X, y in test_loader:
            X = X.to(device)
            y = y.to(device)
            batch_size, num_stocks, input_dim = X.shape
            X_reshaped = X.view(-1, input_dim)
            out = model(X_reshaped, edge_index_prop, edge_weight_prop)
            out = out.view(batch_size, num_stocks, -1)
            predictions.append(out.squeeze().cpu().numpy())
            actuals.append(y.cpu().numpy())
    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)
    rmse = np.sqrt(np.mean((predictions - actuals)**2))
    print(f"Test RMSE: {rmse:.6f}")
    
    r2 = r2_score(actuals, predictions)
    print(f"Test R^2: {r2:.6f}")
    
    pred_sign = np.sign(predictions)
    actual_sign = np.sign(actuals)
    binary_accuracy = np.mean(pred_sign == actual_sign)
    print(f"Test Binary Accuracy: {binary_accuracy:.6f}")
    
    # Print stock names
    print("Stocks:", stocks)
    
    # Save the model for future backtesting
    model_path = os.path.join("saved_models", "stockkan_model.pth")
    os.makedirs("saved_models", exist_ok=True)
    torch.save(model.state_dict(), model_path)
    print(f"Saved model to {model_path}")
    
    # Backtest and generate PnL and statistics for each stock in the dataset,
    # and save portfolio metrics in a text file.
    print("Starting backtesting on test data for all stocks...")
    backtest_all_stocks(model, test_dataset, edge_index_prop, edge_weight_prop, device, output_folder)

if __name__ == '__main__':
    main()


/home/ubuntu/.local/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the m

Saved final propagation graph to graphs/final_cumulative_prop_graph.png
Saved cumulative propagation delays CSV.
Propagation graph edges (tensor): 2094
Using device: cuda


/tmp/ipykernel_146667/27492817.py:486: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/50, Loss: 0.000379


/home/ubuntu/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1, 48])) that is different to the input size (torch.Size([48])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2/50, Loss: 0.000346
Epoch 3/50, Loss: 0.000341
Epoch 4/50, Loss: 0.000361
Epoch 5/50, Loss: 0.000340
Epoch 6/50, Loss: 0.000340
Epoch 7/50, Loss: 0.000336
Epoch 8/50, Loss: 0.000332
Epoch 9/50, Loss: 0.000331
Epoch 10/50, Loss: 0.000326
Epoch 11/50, Loss: 0.000322
Epoch 12/50, Loss: 0.000321
Epoch 13/50, Loss: 0.000306
Epoch 14/50, Loss: 0.000307
Epoch 15/50, Loss: 0.000306
Epoch 16/50, Loss: 0.000305
Epoch 17/50, Loss: 0.000305
Epoch 18/50, Loss: 0.000303
Epoch 19/50, Loss: 0.000301
Epoch 20/50, Loss: 0.000299
Epoch 21/50, Loss: 0.000289
Epoch 22/50, Loss: 0.000286
Epoch 23/50, Loss: 0.000287
Epoch 24/50, Loss: 0.000285
Epoch 25/50, Loss: 0.000280
Epoch 26/50, Loss: 0.000282
Epoch 27/50, Loss: 0.000287
Epoch 28/50, Loss: 0.000278
Epoch 29/50, Loss: 0.000276
Epoch 30/50, Loss: 0.000277
Epoch 31/50, Loss: 0.000282
Epoch 32/50, Loss: 0.000274
Epoch 33/50, Loss: 0.000273
Epoch 34/50, Loss: 0.000269
Epoch 35/50, Loss: 0.000270
Epoch 36/50, Loss: 0.000281
Epoch 37/50, Loss: 0.000272
